In [1]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


PyTorch version: 2.7.1+cpu
CUDA available: False


In [3]:
print("✅ Python works in Jupyter")


✅ Python works in Jupyter


In [4]:
import pandas as pd
import numpy as np
print("✅ pandas & numpy are available:", pd.__version__, np.__version__)


✅ pandas & numpy are available: 2.2.3 2.1.3


In [5]:
import sklearn
print("✅ scikit-learn version:", sklearn.__version__)


✅ scikit-learn version: 1.6.1


In [6]:
# Task 3 - Housing Price Prediction (Pure Python, no external libraries)


# 1. Sample Housing Dataset
#
# Features: [bedrooms, bathrooms, sqft_living]
# Target: price
data = [
    [2, 1, 1000, 150000],
    [3, 2, 1800, 250000],
    [4, 3, 2200, 320000],
    [2, 1, 900, 140000],
    [5, 4, 3000, 450000],
    [3, 2, 1600, 230000],
    [4, 2, 2000, 310000],
    [1, 1, 800, 120000],
    [3, 2, 1700, 240000],
    [5, 3, 2800, 400000],
]

X = [row[:-1] for row in data]  # features
y = [row[-1] for row in data]   # target

# -----------------------
# 2. Train Simple Linear Regression (Normal Equation)
# -----------------------
# Add bias term (1) to each row
X_bias = []
for row in X:
    X_bias.append([1] + row)  # add constant term for intercept

# Matrix operations (implemented manually)
def transpose(M):
    return [list(row) for row in zip(*M)]

def matmul(A, B):
    result = [[0]*len(B[0]) for _ in range(len(A))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j]
    return result

def inverse_2x2(M):
    det = M[0][0]*M[1][1] - M[0][1]*M[1][0]
    return [[M[1][1]/det, -M[0][1]/det],
            [-M[1][0]/det, M[0][0]/det]]

# Convert y to column vector
y_col = [[val] for val in y]

XT = transpose(X_bias)
XTX = matmul(XT, X_bias)


def predict_row(row, weights):
    return sum(w*x for w,x in zip(weights, [1]+row))

def gradient_descent(X, y, lr=0.00000001, epochs=5000):
    weights = [0]*(len(X[0])+1)  # bias + features
    n = len(y)
    for _ in range(epochs):
        grad = [0]*len(weights)
        for xi, yi in zip(X, y):
            pred = predict_row(xi, weights)
            error = pred - yi
            for j,val in enumerate([1]+xi):
                grad[j] += error * val
        # update
        for j in range(len(weights)):
            weights[j] -= lr * grad[j] / n
    return weights

weights = gradient_descent(X, y)
print("Learned weights:", weights)


def mae_rmse(X, y, weights):
    errors = []
    for xi, yi in zip(X, y):
        pred = predict_row(xi, weights)
        errors.append(pred - yi)
    mae = sum(abs(e) for e in errors)/len(errors)
    rmse = (sum(e**2 for e in errors)/len(errors))**0.5
    return mae, rmse

mae, rmse = mae_rmse(X, y, weights)
print(f"✅ MAE: {mae:.2f}")
print(f"✅ RMSE: {rmse:.2f}")

# -----------------------
# 5. Test Predictions
# -----------------------
test_houses = [
    [3, 2, 1500],   # medium house
    [5, 4, 3200],   # big house
    [2, 1, 850],    # small house
]

for house in test_houses:
    price = predict_row(house, weights)
    print(f"House {house} -> Predicted Price: {price:.0f}")


Learned weights: [0.08744244100349388, 0.3050560626028975, 0.1805479181801722, 146.44134964501472]
✅ MAE: 8146.10
✅ RMSE: 9401.68
House [3, 2, 1500] -> Predicted Price: 219663
House [5, 4, 3200] -> Predicted Price: 468615
House [2, 1, 850] -> Predicted Price: 124476
